In [5]:
import numpy as np
import os
CurrentPath = os.getcwd()
data_1 = np.loadtxt(CurrentPath + '/2025/data_1.txt', delimiter=' ')
X = data_1.T[0];Y = data_1.T[1]; var = data_1.T[2]

Metropolis-Hastings algorithm: 
1. Propose a prior

In [30]:
# 0 < a < 1, sigma_a = 0.0005
# -2.6 < b < -2.4, sigma_b = 0.1
# 6.7 < c < 6.9, sigma_c = 0.1
# 16 < d < 18 sigma_d = 1
from numpy import random
a_init = random.uniform(0,1)
b_init = random.uniform(-2.6,  -2.4)
c_init = random.uniform(6.7, 6.9)
d_init = random.uniform(16, 18)



2. calculate the likelihood = chisq

In [7]:
def cube(x,a,b,c,d):
    return a*x**3 + b*x**2 + c*x + d

def chisq(x0,y0,var,a,b,c,d):
    diff = (y0 - cube(x0,a,b,c,d))**2
    chisq = np.sum(diff/var)
    return chisq


3. Create a markov chain

In [17]:
def Markov(paramk,paramkp1,x,y,var):
    minuschisqk = 1/chisq(x,y,var,paramk[0],paramk[1],paramk[2],paramk[3])
    minuschisqkp1 = 1/chisq(x,y,var,paramkp1[0],paramkp1[1],paramkp1[2],paramkp1[3])
    r = np.log(np.random.uniiform(0.,1.))
    if minuschisqk - minuschisqkp1<0:


10.470600485634849

In [24]:
arr = np.zeros((2,3))
arr[:,1] = [1,1]

In [25]:
arr

array([[0., 1., 0.],
       [0., 1., 0.]])